In [1]:
# import libraries

from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
import time

In [2]:
# set up 

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
# collect latest news title and paragraph

url = "https://mars.nasa.gov/news"

browser.visit(url)

soup = BeautifulSoup(browser.html, 'lxml')

title = soup.find('div', class_='content_title')

title = title.text

title

'After a Reset, Curiosity Is Operating Normally'

In [4]:
paragraph = soup.find('div', class_="article_teaser_body")

paragraph = paragraph.text

In [22]:
paragraph

'Curiosity has returned to science operations and is once again exploring the clay unit. '

In [5]:
# assign URL and extract current featured Mars image full size jpg

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

browser.click_link_by_partial_text("FULL IMAGE")

time.sleep(2)

browser.click_link_by_partial_text("more info")

soup = BeautifulSoup(browser.html, 'lxml')

image = soup.find("figure", class_="lede")


In [6]:
image_url = "https://www.jpl.nasa.gov" + image.a['href']

In [23]:
image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18816_hires.jpg'

In [26]:
# scrape latest mars weather tweet

weather_url = "https://twitter.com/marswxreport?lang=en"

browser.visit(weather_url)

soup = BeautifulSoup(browser.html, 'lxml')

tweet = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')


In [61]:
tweet = tweet.text

AttributeError: 'str' object has no attribute 'text'

In [59]:
tweet = tweet.replace('\n', '')

'InSight sol 91 (2019-02-28) low -95.0ºC (-139.0ºF) high -13.9ºC (7.0ºF)winds from the SW at 4.2 m/s (9.5 mph) gusting to 14.0 m/s (31.3 mph)pressure at 7.20 hPapic.twitter.com/yCZi4ZZWYz'

In [8]:
# scrape mars facts table containing diameter, mass, etc

facts_url = "https://space-facts.com/mars"

table = pd.read_html(facts_url)

table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [9]:
df = table[0]

In [10]:
# convert convert data to HTML string table with pandas

html_table = df.to_html()

html_table = html_table.replace('\n', '')

In [11]:
# scrape high res images of each of Mar's hemispheres, save image url and title containing name 

In [50]:
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(hemisphere_url)

soup = BeautifulSoup(browser.html, 'lxml')

hemisphere = []
img_url = [] 

results = soup.find_all('div', class_="description")

for result in results:
    hemisphere.append(result.h3.text)
    browser.visit("https://astrogeology.usgs.gov" + result.a["href"])
    time.sleep(2)
    img_url.append(BeautifulSoup(browser.html, "html.parser").find("div", class_="downloads").a["href"])
    time.sleep(2)

In [51]:
mars = {}

# store in python dictionary using keys img_url and title

# append dictionary with img url string and hemisphere title to list, each list containing one dictionary for each hemisphere



In [52]:
mars["hemisphere"] = {"hemi_title": hemisphere, "img_url": img_url}

In [53]:
mars["news_title"] = title

In [54]:
mars["paragraph"] = paragraph

In [55]:
mars["hemisphere"]["hemi_title"]

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [56]:
mars

{'hemisphere': {'hemi_title': ['Cerberus Hemisphere Enhanced',
   'Schiaparelli Hemisphere Enhanced',
   'Syrtis Major Hemisphere Enhanced',
   'Valles Marineris Hemisphere Enhanced'],
  'img_url': ['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']},
 'news_title': 'After a Reset, Curiosity Is Operating Normally',
 'paragraph': 'Curiosity has returned to science operations and is once again exploring the clay unit. '}